## Hateful Tweet Selection

This notebook implements selection of hateful tweets from the annotated tweets. The selection is based on the following criteria:
- The tweet is annotated as hateful against a target group by three annotators
- The tweet is Turkish 

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('../data/all_tweets.csv')
df

,genel_tutum,hedef_grup,derece,kategori,saldırgan_dil,tweet_text,annotator,uploaded_at,konu,tweet_id,dil
0,Nötr,['Hedef grup belirgin değil veya yok.'],0,['Nefret söylemi bulunmuyor'],Yok,'Tûr Dağını yaşa\nKi bilesin nerde Kudüs\nBen ...,Batuhan,2022-12-30T17:14:27,Isr-Pal,1390387083638939650,NaN
1,Alakasız,['Hedef grup belirgin değil veya yok.'],0,['Nefret söylemi bulunmuyor'],Yok,@hurmafidesi #KuduesBizimdir \n#KudusBizimdir,Batuhan,2022-12-30T17:14:27,Isr-Pal,1390429568666853376,NaN
2,Yahudi Karşıtı,['Ülke/Milliyet'],5,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,#Kahrolsunİsrail https://t.co/p03lUXHEqH,Batuhan,2022-12-30T17:14:27,Isr-Pal,1390788028084465666,NaN
3,Nötr,['Hedef grup belirgin değil veya yok.'],0,['Nefret söylemi bulunmuyor'],Yok,#KudüsFilistininBaşkentidir\n#KudüsBizimdir ht...,Batuhan,2022-12-30T17:14:27,Isr-Pal,1390984251840024576,NaN
4,Alakasız,['Hedef grup belirgin değil veya yok.'],0,['Nefret söylemi bulunmuyor'],Yok,Carlos Manucci Trujillo\n0 - 0\nAyacucho FC \...,Batuhan,2022-12-30T17:14:27,Isr-Pal,1391167506715398145,NaN
...,...,...,...,...,...,...,...,...,...,...,...
47627,Alakasız,['Hedef grup belirgin değil veya yok.'],4,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,@solcugazete Olum bu sapkın admine nasıl inanı...,Firdevs,2023-11-27T09:35:59,LGBTI+,1658577526690663936,Türkçe
47628,Alakasız,['Irk/Etnik Köken'],8,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Şiddetli,@ayseerdogan06tr Sizi 15 milyon mülteci rahats...,Firdevs,2023-11-27T09:35:59,LGBTI+,1658599575836392960,Türkçe
47629,Alakasız,['Din'],6,"['Simgeleştirme', 'Küfür; Hakaret; Aşağılama; ...",Zayıf,@BeyzaNehir99 @feridevmi Güzel ablam inanma bu...,Firdevs,2023-11-27T09:35:59,LGBTI+,1658667235584377088,Türkçe
47630,Alakasız,['Belli Görüş/Statü/Uygulama; Mesleki Pozisyon...,5,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,@nefise123456789 @wrholia Lan çocuksun amk çoc...,Firdevs,2023-11-27T09:35:59,LGBTI+,1658709293758045952,Türkçe


In [3]:
df['genel_tutum'].value_counts()

Nötr                      12970
Alakasız                   7641
Irrelevant                 5545
Göçmen/Mülteci Karşıtı     5278
Yunan Karşıtı              3397
Yahudi Karşıtı             3108
Emin değilim               2262
Ermeni Karşıtı             1656
Arap Karşıtı               1097
Anti-Immigrant/Refugee      824
Neutral                     392
Alevi Karşıtı               381
Not Sure                    354
Türkçe değil                265
Kürt Karşıtı                232
LGBTI+ Karşıtı              197
Name: genel_tutum, dtype: int64

In [4]:
df.drop_duplicates(subset='tweet_id')['genel_tutum'].value_counts() 

Nötr                      4125
Alakasız                  2585
Irrelevant                2347
Göçmen/Mülteci Karşıtı    1821
Yunan Karşıtı             1283
Yahudi Karşıtı            1091
Emin değilim               929
Ermeni Karşıtı             620
Arap Karşıtı               449
Anti-Immigrant/Refugee     280
Neutral                    153
Not Sure                   120
Alevi Karşıtı              111
LGBTI+ Karşıtı              98
Kürt Karşıtı                89
Türkçe değil                85
Name: genel_tutum, dtype: int64

In [5]:
# Function to keep groups with at least three rows and have the same value in genel_tutum
def process_group(group):
    if len(group) == 3 and group['genel_tutum'].nunique() == 1:
        return group
    else:
        print(group['tweet_id'].values[0], 'is not a consensus group', group['genel_tutum'].values)
        return None
df_consensus = df.groupby('tweet_id').apply(process_group).reset_index(drop=True)
df_consensus

1353049567248457729 is not a consensus group ['Yahudi Karşıtı' 'Yahudi Karşıtı' 'Nötr']
1353223622958182403 is not a consensus group ['Nötr' 'Yahudi Karşıtı' 'Nötr']
1353611898026156032 is not a consensus group ['Yahudi Karşıtı' 'Yahudi Karşıtı' 'Emin değilim']
1356275317271371776 is not a consensus group ['Yahudi Karşıtı' 'Nötr' 'Yahudi Karşıtı']
1366346815109533698 is not a consensus group ['Nötr' 'Yahudi Karşıtı' 'Yahudi Karşıtı']
1368970739425087490 is not a consensus group ['Nötr' 'Nötr' 'Yahudi Karşıtı']
1369715338502373377 is not a consensus group ['Yahudi Karşıtı' 'Yahudi Karşıtı' 'Nötr']
1378503816526639113 is not a consensus group ['Nötr' 'Emin değilim' 'Yahudi Karşıtı']
1380813010575429634 is not a consensus group [nan nan nan]
1382615665790619649 is not a consensus group ['Yahudi Karşıtı' 'Nötr' 'Nötr']
1382832299268530178 is not a consensus group ['Yahudi Karşıtı' 'Nötr' 'Yahudi Karşıtı']
1383010960588345345 is not a consensus group ['Yahudi Karşıtı' 'Yahudi Karşıtı' 'Nötr

,genel_tutum,hedef_grup,derece,kategori,saldırgan_dil,tweet_text,annotator,uploaded_at,konu,tweet_id,dil
0,Yahudi Karşıtı,['Ülke/Milliyet'],7,['Abartma; Genelleme; Yükleme; Çarpıtma'],Şiddetli,#Katilİsrail https://t.co/A2uX5Uygxs,Ebrar,2023-01-10T19:59:49,Isr-Pal,1353048938450976770,NaN
1,Yahudi Karşıtı,['Ülke/Milliyet'],4,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,#Katilİsrail https://t.co/A2uX5Uygxs,Göksu,2023-01-10T20:07:20,Isr-Pal,1353048938450976770,NaN
2,Yahudi Karşıtı,['Ülke/Milliyet'],6,['Düşmanlık; Savaş; Saldırı; Öldürme; Yaralama...,Yok,#Katilİsrail https://t.co/A2uX5Uygxs,Ceren,2023-01-10T19:48:53,Isr-Pal,1353048938450976770,NaN
3,Yahudi Karşıtı,['Ülke/Milliyet'],7,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Şiddetli,"@veliagbaba Suriyenin iç işlerine karışma, ora...",Ebrar,2023-02-01T17:44:11,Isr-Pal,1355236074201751554,Türkçe
4,Yahudi Karşıtı,['Ülke/Milliyet'],3,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,"@veliagbaba Suriyenin iç işlerine karışma, ora...",Altar,2023-02-09T17:51:12,Isr-Pal,1355236074201751554,Türkçe
...,...,...,...,...,...,...,...,...,...,...,...
18754,Irrelevant,['Target group is unclear or absent'],0,['There is no hate speech'],None,ما لا يتم الواجب إلا به فهو #واجب \nانتخاب رجب...,Amr Subbah,2023-08-03T15:08:53,Arabic,1660064644889755651,NaN
18755,Irrelevant,"['Religion', 'Specific Opinion/Status/Practice...",2,['Symbolization'],Low,ما لا يتم الواجب إلا به فهو #واجب \nانتخاب رجب...,Majd,2024-01-20T09:59:59,Arabic,1660064644889755651,NaN
18756,Irrelevant,['Target group is unclear or absent'],0,['There is no hate speech'],None,من حسن حظك سعادة السفيرة\nأنك تشرفين على أهم إ...,Neda,2023-11-14T07:57:32,Arabic,1660072583054737408,NaN
18757,Irrelevant,['Target group is unclear or absent'],0,['There is no hate speech'],None,من حسن حظك سعادة السفيرة\nأنك تشرفين على أهم إ...,Amr Subbah,2024-03-03T08:04:44,Arabic,1660072583054737408,NaN


In [6]:
df_consensus.drop_duplicates(subset='tweet_id')['genel_tutum'].value_counts() 

Göçmen/Mülteci Karşıtı    1416
Nötr                      1242
Irrelevant                1047
Yunan Karşıtı              603
Alakasız                   522
Yahudi Karşıtı             402
Ermeni Karşıtı             325
Arap Karşıtı               155
Anti-Immigrant/Refugee     102
LGBTI+ Karşıtı              28
Türkçe değil                18
Kürt Karşıtı                17
Emin değilim                11
Neutral                     11
Alevi Karşıtı               11
Name: genel_tutum, dtype: int64

In [8]:
df_consensus[df_consensus['genel_tutum'].isin(['Nötr', 'Alakasız'])].drop_duplicates(subset='tweet_id').to_csv('../data/nonhateful_tweets.csv', index=False)

In [7]:
target_groups = [
    'Göçmen/Mülteci Karşıtı', 
    'Yunan Karşıtı', 
    'Yahudi Karşıtı', 
    'Ermeni Karşıtı', 'Arap Karşıtı', 
    'Anti-Immigrant/Refugee',
    'LGBTI+ Karşıtı',
    'Kürt Karşıtı',
    'Alevi Karşıtı'
]

In [8]:
from ast import literal_eval
import numpy as np
df_consensus['kategori'] = df_consensus['kategori'].apply(lambda s: literal_eval(s) if s is not np.nan else [])

In [9]:
df_consensus['kategori_count'] = df_consensus['kategori'].apply(len)
df_consensus['kategori_count'].value_counts()

1    14144
2     2984
0      807
3      659
4      151
5       14
Name: kategori_count, dtype: int64

In [10]:
df_consensus_target = df_consensus[df_consensus['genel_tutum'].isin(target_groups)]
df_consensus_target['kategori_count'].value_counts()

1    5685
2    2767
3     643
4     146
5      14
0       2
Name: kategori_count, dtype: int64

In [16]:
# Load the earlier samples to exclude them from the new samples
samples_v1 = pd.read_csv('../data/span_samples.csv')
samples_v2 = pd.read_csv('../data/span_samples_v2.csv')

df_consensus_target = df_consensus_target[~df_consensus_target['tweet_id'].isin(samples_v1['tweet_id'])]
df_consensus_target = df_consensus_target[~df_consensus_target['tweet_id'].isin(samples_v2['tweet_id'])]

In [17]:
# sample 50 tweets from each group
samples = df_consensus_target.drop_duplicates(subset='tweet_id').groupby('genel_tutum').apply(lambda group: group.sample(50) if len(group) > 50 else group).reset_index(drop=True)
samples['genel_tutum'].value_counts()

Arap Karşıtı              50
Ermeni Karşıtı            50
Göçmen/Mülteci Karşıtı    50
Yahudi Karşıtı            50
Yunan Karşıtı             50
Anti-Immigrant/Refugee     2
Name: genel_tutum, dtype: int64

In [18]:
samples.to_csv('../data/span_samples_v3.csv', index=False)

In [15]:
# find common elements across several lists
from functools import reduce
def find_common_elements(lists):
    try:
        return list(reduce(lambda x, y: set(x) & set(y), lists)) 
    except:
        return []

In [16]:
def process_group(group):
    #print(group['kategori'], find_common_elements(group['kategori']) )
    group['kategori_common'] = [find_common_elements(group['kategori'])] * len(group)
    if len(group) == 3 and group['kategori_str'].nunique() == 1:
    #if len(group) == 3 and len(group['kategori_common']) > 0:
        return group
    else:
        return None
df_consensus_target['kategori_str'] = df_consensus_target['kategori'].apply(lambda x: ', '.join(x))
df_consensus_target_cat = df_consensus_target.groupby('tweet_id').apply(process_group).reset_index(drop=True)
df_consensus_target_cat

C:\Users\gokce\AppData\Local\Temp\ipykernel_4164\2131976881.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consensus_target['kategori_str'] = df_consensus_target['kategori'].apply(lambda x: ', '.join(x))


,genel_tutum,hedef_grup,derece,kategori,saldırgan_dil,tweet_text,annotator,uploaded_at,konu,tweet_id,dil,kategori_count,kategori_str,kategori_common
0,Yahudi Karşıtı,['Ülke/Milliyet'],5,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Zayıf,Öfkeni göster dostum öfken onların korkulu rüy...,Ebrar,2023-02-01T17:44:11,Isr-Pal,1390845435602931713,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
1,Yahudi Karşıtı,['Ülke/Milliyet'],5,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Zayıf,Öfkeni göster dostum öfken onların korkulu rüy...,Göksu,2023-03-13T07:19:49,Isr-Pal,1390845435602931713,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
2,Yahudi Karşıtı,['Irk/Etnik Köken'],5,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Yok,Öfkeni göster dostum öfken onların korkulu rüy...,Mehmet,2023-02-01T17:46:00,Isr-Pal,1390845435602931713,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
3,Yahudi Karşıtı,['Ülke/Milliyet'],2,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Zayıf,#Kahrolsunİsrail \n#MescidiAksaYanlızDeğildir ...,Aygün,2023-03-21T13:17:12,Isr-Pal,1390959516582285313,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
4,Yahudi Karşıtı,['Ülke/Milliyet'],5,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Yok,#Kahrolsunİsrail \n#MescidiAksaYanlızDeğildir ...,Nur,2023-03-06T10:36:27,Isr-Pal,1390959516582285313,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,Ermeni Karşıtı,['Irk/Etnik Köken'],5,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Zayıf,"@REM__BEN @herkesicinCHP Kılıçtaroğlu, imamoğl...",Nur,2023-09-08T10:48:34,Armenian,1658546689387421952,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
1115,Ermeni Karşıtı,"['Irk/Etnik Köken', 'Ülke/Milliyet']",4,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma],Zayıf,"@REM__BEN @herkesicinCHP Kılıçtaroğlu, imamoğl...",Deniz,2023-09-13T10:41:00,Armenian,1658546689387421952,Türkçe,1,Küfür; Hakaret; Aşağılama; İnsandışılaştırma,[Küfür; Hakaret; Aşağılama; İnsandışılaştırma]
1116,Kürt Karşıtı,['Irk/Etnik Köken'],3,[Abartma; Genelleme; Yükleme; Çarpıtma],Zayıf,@SimonettaVspc Filiz senin Kürt sorununu da se...,Şura,2023-12-12T01:41:13,Kurdish,1658603760451571968,Türkçe,1,Abartma; Genelleme; Yükleme; Çarpıtma,[Abartma; Genelleme; Yükleme; Çarpıtma]
1117,Kürt Karşıtı,"['Irk/Etnik Köken', 'Belli Görüş/Statü/Uygulam...",4,[Abartma; Genelleme; Yükleme; Çarpıtma],Zayıf,@SimonettaVspc Filiz senin Kürt sorununu da se...,Hilal,2023-12-12T01:41:22,Kurdish,1658603760451571968,Türkçe,1,Abartma; Genelleme; Yükleme; Çarpıtma,[Abartma; Genelleme; Yükleme; Çarpıtma]
